In [1]:
# Imports
import sqlite3
import time

from graphviz import Graph

from pyalgdb.validity import Validity 
from pyalgdb.node import Node
from pyalgdb.provenance_enhancement import ProvenanceEnhancement
from pyalgdb.execution_tree_creator import ExecTreeCreator
from pyalgdb.top_down import TopDown
from pyalgdb.heaviest_first import HeaviestFirst
from pyalgdb.visualization import Visualization
from pyalgdb.single_stepping import SingleStepping
from pyalgdb.divide_and_query import DivideAndQuery

In [2]:
NAMES = ['age-avg', 'min-max', 's-example', 'carstore', 'text-processing']
EXAMPLE_NAME = NAMES[2]

In [3]:
NOW2_SQLITE_PATH = 'C:/Users/linha/Desktop/ws/py-scripts-examples/' + EXAMPLE_NAME + '/.noworkflow/db.sqlite'

In [4]:
ANSWER_FILE_PATH = 'aux_files/' + EXAMPLE_NAME + '-answers.json'

In [6]:
GENERATE_TREES = True

In [7]:
RUN_1ST_EXPERIMENT = True
RUN_2ND_EXPERIMENT = True
RUN_3RD_EXPERIMENT = True

In [8]:
CURSOR = sqlite3.connect(NOW2_SQLITE_PATH).cursor()

In [9]:
creator = ExecTreeCreator(CURSOR)

In [10]:
#################################
# FIRST EXPERIMENT
# COMPARING NAVIGATION STRATEGIES WITHOUT PROVENANCE

In [11]:
navs = [SingleStepping, TopDown, HeaviestFirst, DivideAndQuery] 

In [12]:
if RUN_1ST_EXPERIMENT:
    for nav in navs:
        exec_tree = None
        exec_tree = creator.create_exec_tree()
        nav_instance = nav(exec_tree, True, ANSWER_FILE_PATH)
        nav_instance.navigate()
        print(nav_instance.__class__.__name__+" experiment finished: "+str(nav_instance.sequence_num)+" steps.")
        if GENERATE_TREES:
            vis = Visualization(exec_tree)
            vis.view_exec_tree(str(id(exec_tree)))
    

SingleStepping experiment finished: 4 steps.
TopDown experiment finished: 4 steps.
HeaviestFirst experiment finished: 5 steps.
DivideAndQuery experiment finished: 4 steps.


In [13]:
#################################
# SECOND EXPERIMENT
# COMPARING NAVIGATION STRATEGIES WITH PROVENANCE PRUNE, BUT WITHOUT ASKING WHICH OUTPUT DATA IS WRONG

In [14]:
navs = [SingleStepping, TopDown, HeaviestFirst, DivideAndQuery] 

In [15]:
if RUN_2ND_EXPERIMENT:
    for nav in navs:
        exec_tree = None
        exec_tree = creator.create_exec_tree()
        prov = ProvenanceEnhancement(exec_tree, CURSOR)
        prov.enhance_all()    
        nav_instance = nav(exec_tree, True, ANSWER_FILE_PATH)
        nav_instance.provenance_prune()
        nav_instance.navigate()
        print(nav_instance.__class__.__name__+" experiment finished: "+str(nav_instance.sequence_num)+" steps.")
        if GENERATE_TREES:
            vis = Visualization(exec_tree)
            vis.view_exec_tree(str(id(exec_tree)))

SingleStepping experiment finished: 3 steps.
TopDown experiment finished: 4 steps.
HeaviestFirst experiment finished: 5 steps.
DivideAndQuery experiment finished: 4 steps.


In [16]:
#################################
# THIRD EXPERIMENT
# COMPARING NAVIGATION STRATEGIES WITH PROVENANCE PRUNE, ASKING WHICH OUTPUT DATA IS WRONG

In [17]:
navs = [SingleStepping, TopDown, HeaviestFirst, DivideAndQuery] 

In [18]:
if RUN_3RD_EXPERIMENT:
    for nav in navs:
        exec_tree = None
        exec_tree = creator.create_exec_tree()
        nav_instance = nav(exec_tree, True, ANSWER_FILE_PATH)
        prov = ProvenanceEnhancement(exec_tree, CURSOR)
        wrong_node_ev = exec_tree.search_by_ev_id(nav_instance.wrong_node_id)
        prov.enhance(wrong_node_ev)    
        nav_instance.provenance_prune()
        nav_instance.navigate()
        print(nav_instance.__class__.__name__+" experiment finished: "+str(nav_instance.sequence_num)+" steps.")
        if GENERATE_TREES:
            vis = Visualization(exec_tree)
            vis.view_exec_tree(str(id(exec_tree)))

SingleStepping experiment finished: 3 steps.
TopDown experiment finished: 3 steps.
HeaviestFirst experiment finished: 5 steps.
DivideAndQuery experiment finished: 3 steps.
